# Champion Mastery - Europe

In [46]:
import numpy as np
import pandas as pd
import requests as re
import time

In [49]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36 Edg/103.0.1264.77",
    "Accept-Language": "en-CA,en-US;q=0.9,en;q=0.8",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": "RGAPI-046cac00-eb22-432a-94e1-9c68bd571a92"
}

In [50]:
EU = pd.read_csv('../../data/first_model/raw/EUW_all_data_366.csv')
EU.head()

,SERVER,MATCH_ID,BSUM1_ID,BTOP_CHAMP_ID,BSUM2_ID,BJGL_CHAMP_ID,BSUM3_ID,BMID_CHAMP_ID,BSUM4_ID,BBOT_CHAMP_ID,...,FIRST_TOWER,BTOWER_KILLS,RTOWER_KILLS,FIRST_BARON,BBARON_KILLS,RBARON_KILLS,FIRST_INHIB,BINHIB_KILLS,RINHIB_KILLS,WINNER
0,EUROPE WEST,EUW1_6001811167,0AZy9ZPtWQh6Vx8Kq7OvFy4UrZwrJACKWYJs-E6rzV4cizM,150,xh8g794rjw2i3e2OFWyH8NLvm9U94qaGNIAvSZUTDC9-BJRL,246,9T_e3hPA8BJJkJ-VGvGVJWR24xPZ0yyY5aGTOIKl95Cd6h8g,268,7Fa7S45GBBFAvaW3BGMq4AJpClEZ61ZamQCUPYjGAgBkxRg,15,...,RED,2,4,NONE,0,0,NONE,0,0,RED
1,EUROPE WEST,EUW1_6001677114,o02MjualoBd8c2L1Nu2ZRjnhkt_ne-Yx2iYRZkhd3uDKsY4,114,CaqfcvprY1V7DVI5sfOJP5Vc3luQoyshL8AemHrvh43qUQw,254,puPeQuN3cAKued6EAm3b5gd6Al11zyzbm32lGMki4yemKt2x,79,jWysZe-gwuPtokIJR-NI8cCCG-GninDNH-TpdxiJ9r-ZaxA,498,...,RED,0,4,NONE,0,0,NONE,0,0,RED
2,EUROPE WEST,EUW1_6001722049,lz7z9m2ZwFuDm0kSa-6SYXtXLFTLLLlWizzRxSyCgh-Msk0,150,Gu0-dhJT1_53zm_TQt4BQ1uQ_CZhl4fHrN33OQG3cfz5teM,78,_o_3E5f4P_CY9ztsrbdkUPwVCg7hJ7TSAXHiEEsevv1bdVbI,517,YYnE90RJfzypVDAO_Qt8syFQwCwhunwTWEfidwcucoFPqtc,523,...,RED,5,8,RED,0,1,RED,1,2,BLUE
3,EUROPE WEST,EUW1_6001557446,ZCHr8rokYRXJMJIUWzFcIehrJs6jShr0Zkc3aFJZ4xEiTfE,79,Gu0-dhJT1_53zm_TQt4BQ1uQ_CZhl4fHrN33OQG3cfz5teM,80,sO2INLctOSFuq_VBeqSjSAC6faH4PvvgZA3grfCOathsYUSH,4,hgowSBUFsyDNo-S2AshXWRF2xM_mOBktpCtV1Mw7K4sRIq0,18,...,BLUE,6,4,RED,0,1,NONE,0,0,RED
4,EUROPE WEST,EUW1_6001602687,Yp9kCQ3SqtxY-dyRdyAm7i3zZgtduT21MlAjGPN_0NZ5tR...,122,Gu0-dhJT1_53zm_TQt4BQ1uQ_CZhl4fHrN33OQG3cfz5teM,80,hIoSka6anKc6_WSUhIDk06K6JnZF5kz5JaeSaKeLIA0U8m...,517,BgbTbMspKYR1yIsQjzW-e_bpn45fjAEcAPnAGX5dLfR6sao,96,...,BLUE,7,2,NONE,0,0,BLUE,1,0,BLUE


In [51]:
EU.shape

(366, 51)

In [52]:
EU1 = EU[:100]
EU2 = EU[100:200]
EU3 = EU[200:300]
EU4 = EU[300:]

In [53]:
'''
This function uses the champion mastery api to get the champion mastery for a specific champion and summoner.

Region can be: euw1, na1, kr
Summoner input can be:
-  BSUM1_ID - BSUM5_ID
-  RSUM1_ID - RSUM5_ID

Summoner_champ_id input can be:
- BTOP_CHAMP_ID or RTOP_CHAMP_ID
- BJGL_CHAMP_ID or RJGL_CHAMP_ID
- BMID_CHAMP_ID or RMID_CHAMP_ID
- BBOT_CHAMP_ID or RBOT_CHAMP_ID
- BSUP_CHAMP_ID or RSUP_CHAMP_ID
'''


def champion_mastery(df, server, summoner, summoner_champ_id):

    mastery_list = []
    for i in range(len(df)):
        summoner_id = df.iloc[i][summoner]
        champion_id = df.iloc[i][summoner_champ_id]

        try:
            url = 'https://'+ server + '.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/' + summoner_id + '/by-champion/' + str(champion_id)
            res = re.get(url, headers=headers).json()

            mastery = res['championPoints']
            mastery_list.append(mastery)
        except:
            mastery_list.append('error')
            pass

    return mastery_list


## Blue Top

In [57]:
Blue_top_list1 = champion_mastery(EU1, 'euw1', 'BSUM1_ID', 'BTOP_CHAMP_ID')
time.sleep(120)
Blue_top_list2 = champion_mastery(EU2, 'euw1', 'BSUM1_ID', 'BTOP_CHAMP_ID')
time.sleep(120)
Blue_top_list3 = champion_mastery(EU3, 'euw1', 'BSUM1_ID', 'BTOP_CHAMP_ID')
time.sleep(120)
Blue_top_list4 = champion_mastery(EU4, 'euw1', 'BSUM1_ID', 'BTOP_CHAMP_ID')
time.sleep(120)
Blue_TOP_list = Blue_top_list1 + Blue_top_list2 + Blue_top_list3 + Blue_top_list4
len(Blue_TOP_list)

366

In [59]:
if 'error' in Blue_TOP_list:
    print(True)
else:
    print(False)

False


## Blue JGL

In [60]:
Blue_JGL_list1 = champion_mastery(EU1, 'euw1', 'BSUM2_ID', 'BJGL_CHAMP_ID')
time.sleep(120)
Blue_JGL_list2 = champion_mastery(EU2, 'euw1', 'BSUM2_ID', 'BJGL_CHAMP_ID')
time.sleep(120)
Blue_JGL_list3 = champion_mastery(EU3, 'euw1', 'BSUM2_ID', 'BJGL_CHAMP_ID')
time.sleep(120)
Blue_JGL_list4 = champion_mastery(EU4, 'euw1', 'BSUM2_ID', 'BJGL_CHAMP_ID')
time.sleep(120)
Blue_JGL_list = Blue_JGL_list1 + Blue_JGL_list2 + Blue_JGL_list3 + Blue_JGL_list4
len(Blue_JGL_list)

366

In [61]:
if 'error' in Blue_JGL_list:
    print(True)
else:
    print(False)

False


## Blue MID

In [62]:
Blue_MID_list1 = champion_mastery(EU1, 'euw1', 'BSUM3_ID', 'BMID_CHAMP_ID')
time.sleep(120)
Blue_MID_list2 = champion_mastery(EU2, 'euw1', 'BSUM3_ID', 'BMID_CHAMP_ID')
time.sleep(120)
Blue_MID_list3 = champion_mastery(EU3, 'euw1', 'BSUM3_ID', 'BMID_CHAMP_ID')
time.sleep(120)
Blue_MID_list4 = champion_mastery(EU4, 'euw1', 'BSUM3_ID', 'BMID_CHAMP_ID')
time.sleep(120)
Blue_MID_list = Blue_MID_list1 + Blue_MID_list2 + Blue_MID_list3 + Blue_MID_list4
len(Blue_MID_list)

366

In [63]:
if 'error' in Blue_MID_list:
    print(True)
else:
    print(False)

False


## Blue BOT

In [64]:
Blue_BOT_list1 = champion_mastery(EU1, 'euw1', 'BSUM4_ID', 'BBOT_CHAMP_ID')
time.sleep(120)
Blue_BOT_list2 = champion_mastery(EU2, 'euw1', 'BSUM4_ID', 'BBOT_CHAMP_ID')
time.sleep(120)
Blue_BOT_list3 = champion_mastery(EU3, 'euw1', 'BSUM4_ID', 'BBOT_CHAMP_ID')
time.sleep(120)
Blue_BOT_list4 = champion_mastery(EU4, 'euw1', 'BSUM4_ID', 'BBOT_CHAMP_ID')
time.sleep(120)
Blue_BOT_list = Blue_BOT_list1 + Blue_BOT_list2 + Blue_BOT_list3 + Blue_BOT_list4
len(Blue_BOT_list)

366

In [65]:
if 'error' in Blue_BOT_list:
    print(True)
else:
    print(False)

False


## Blue SUP

In [66]:
Blue_SUP_list1 = champion_mastery(EU1, 'euw1', 'BSUM5_ID', 'BSUP_CHAMP_ID')
time.sleep(120)
Blue_SUP_list2 = champion_mastery(EU2, 'euw1', 'BSUM5_ID', 'BSUP_CHAMP_ID')
time.sleep(120)
Blue_SUP_list3 = champion_mastery(EU3, 'euw1', 'BSUM5_ID', 'BSUP_CHAMP_ID')
time.sleep(120)
Blue_SUP_list4 = champion_mastery(EU4, 'euw1', 'BSUM5_ID', 'BSUP_CHAMP_ID')
time.sleep(120)
Blue_SUP_list = Blue_SUP_list1 + Blue_SUP_list2 + Blue_SUP_list3 + Blue_SUP_list4
len(Blue_SUP_list)

366

In [67]:
if 'error' in Blue_SUP_list:
    print(True)
else:
    print(False)

False


## Red TOP

In [68]:
Red_TOP_list1 = champion_mastery(EU1, 'euw1', 'RSUM1_ID', 'RTOP_CHAMP_ID')
time.sleep(120)
Red_TOP_list2 = champion_mastery(EU2, 'euw1', 'RSUM1_ID', 'RTOP_CHAMP_ID')
time.sleep(120)
Red_TOP_list3 = champion_mastery(EU3, 'euw1', 'RSUM1_ID', 'RTOP_CHAMP_ID')
time.sleep(120)
Red_TOP_list4 = champion_mastery(EU4, 'euw1', 'RSUM1_ID', 'RTOP_CHAMP_ID')
time.sleep(120)
Red_TOP_list = Red_TOP_list1 + Red_TOP_list2 + Red_TOP_list3 + Red_TOP_list4
len(Red_TOP_list)

366

In [69]:
if 'error' in Red_TOP_list:
    print(True)
else:
    print(False)

False


## Red JGL

In [70]:
Red_JGL_list1 = champion_mastery(EU1, 'euw1', 'RSUM2_ID', 'RJGL_CHAMP_ID')
time.sleep(120)
Red_JGL_list2 = champion_mastery(EU2, 'euw1', 'RSUM2_ID', 'RJGL_CHAMP_ID')
time.sleep(120)
Red_JGL_list3 = champion_mastery(EU3, 'euw1', 'RSUM2_ID', 'RJGL_CHAMP_ID')
time.sleep(120)
Red_JGL_list4 = champion_mastery(EU4, 'euw1', 'RSUM2_ID', 'RJGL_CHAMP_ID')
time.sleep(120)
Red_JGL_list = Red_JGL_list1 + Red_JGL_list2 + Red_JGL_list3 + Red_JGL_list4
len(Red_JGL_list)

366

In [71]:
if 'error' in Red_TOP_list:
    print(True)
else:
    print(False)

False


## Red MID

In [72]:
Red_MID_list1 = champion_mastery(EU1, 'euw1', 'RSUM3_ID', 'RMID_CHAMP_ID')
time.sleep(120)
Red_MID_list2 = champion_mastery(EU2, 'euw1', 'RSUM3_ID', 'RMID_CHAMP_ID')
time.sleep(120)
Red_MID_list3 = champion_mastery(EU3, 'euw1', 'RSUM3_ID', 'RMID_CHAMP_ID')
time.sleep(120)
Red_MID_list4 = champion_mastery(EU4, 'euw1', 'RSUM3_ID', 'RMID_CHAMP_ID')
time.sleep(120)
Red_MID_list = Red_MID_list1 + Red_MID_list2 + Red_MID_list3 + Red_MID_list4
len(Red_MID_list)

366

In [73]:
if 'error' in Red_MID_list:
    print(True)
else:
    print(False)

False


## Red BOT

In [74]:
Red_BOT_list1 = champion_mastery(EU1, 'euw1', 'RSUM4_ID', 'RBOT_CHAMP_ID')
time.sleep(120)
Red_BOT_list2 = champion_mastery(EU2, 'euw1', 'RSUM4_ID', 'RBOT_CHAMP_ID')
time.sleep(120)
Red_BOT_list3 = champion_mastery(EU3, 'euw1', 'RSUM4_ID', 'RBOT_CHAMP_ID')
time.sleep(120)
Red_BOT_list4 = champion_mastery(EU4, 'euw1', 'RSUM4_ID', 'RBOT_CHAMP_ID')
time.sleep(120)
Red_BOT_list = Red_BOT_list1 + Red_BOT_list2 + Red_BOT_list3 + Red_BOT_list4
len(Red_BOT_list)

366

In [75]:
if 'error' in Red_BOT_list:
    print(True)
else:
    print(False)

False


## Red SUP

In [76]:
Red_SUP_list1 = champion_mastery(EU1, 'euw1', 'RSUM5_ID', 'RSUP_CHAMP_ID')
time.sleep(120)
Red_SUP_list2 = champion_mastery(EU2, 'euw1', 'RSUM5_ID', 'RSUP_CHAMP_ID')
time.sleep(120)
Red_SUP_list3 = champion_mastery(EU3, 'euw1', 'RSUM5_ID', 'RSUP_CHAMP_ID')
time.sleep(120)
Red_SUP_list4 = champion_mastery(EU4, 'euw1', 'RSUM5_ID', 'RSUP_CHAMP_ID')
time.sleep(120)
Red_SUP_list = Red_SUP_list1 + Red_SUP_list2 + Red_SUP_list3 + Red_SUP_list4
len(Red_SUP_list)

366

In [77]:
if 'error' in Red_SUP_list:
    print(True)
else:
    print(False)

False


## Champion Mastery Dataset

In [78]:
EU_mastery = pd.DataFrame()
EU_mastery['BTOP_CHAMP_MAST'] = Blue_TOP_list
EU_mastery['BJGL_CHAMP_MAST'] = Blue_JGL_list
EU_mastery['BMID_CHAMP_MAST'] = Blue_MID_list
EU_mastery['BBOT_CHAMP_MAST'] = Blue_BOT_list
EU_mastery['BSUP_CHAMP_MAST'] = Blue_SUP_list
EU_mastery['RTOP_CHAMP_MAST'] = Red_TOP_list
EU_mastery['RJGL_CHAMP_MAST'] = Red_JGL_list
EU_mastery['RMID_CHAMP_MAST'] = Red_MID_list
EU_mastery['RBOT_CHAMP_MAST'] = Red_BOT_list
EU_mastery['RSUP_CHAMP_MAST'] = Red_SUP_list
EU_mastery.head()

,BTOP_CHAMP_MAST,BJGL_CHAMP_MAST,BMID_CHAMP_MAST,BBOT_CHAMP_MAST,BSUP_CHAMP_MAST,RTOP_CHAMP_MAST,RJGL_CHAMP_MAST,RMID_CHAMP_MAST,RBOT_CHAMP_MAST,RSUP_CHAMP_MAST
0,46397,387356,135748,102169,266708,707714,43375,16727,134518,919414
1,1161674,35904,654979,3998,247984,551271,48145,735792,102169,67304
2,156915,48145,111733,177752,46577,27298,285117,5336,136596,654979
3,82980,43375,315491,167949,41931,10929,12813,919414,105808,48602
4,462466,43375,40043,758251,240933,654979,2669483,62243,39954,397205


In [ ]:
EU_mastery.to_csv('EU_mastery_366.csv', index=False)

In [82]:
EU_mastery.isin(["error"]).any()

BTOP_CHAMP_MAST    False
BJGL_CHAMP_MAST    False
BMID_CHAMP_MAST    False
BBOT_CHAMP_MAST    False
BSUP_CHAMP_MAST    False
RTOP_CHAMP_MAST    False
RJGL_CHAMP_MAST    False
RMID_CHAMP_MAST    False
RBOT_CHAMP_MAST    False
RSUP_CHAMP_MAST    False
dtype: bool